<a href="https://colab.research.google.com/github/catmalark/Tensorflow-Study-Notes/blob/main/WaterPotabilityClassificationNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import layers

In [7]:
data = pd.read_csv("/content/water_potability.csv")

data = data.dropna(axis=0)
data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000
mean,7.085990,195.968072,21917.441374,7.134338,333.224672,426.526409,14.357709,66.400859,3.969729,0.403282
std,1.573337,32.635085,8642.239815,1.584820,41.205172,80.712572,3.324959,16.077109,0.780346,0.490678
min,0.227499,73.492234,320.942611,1.390871,129.000000,201.619737,2.200000,8.577013,1.450000,0.000000
25%,6.089723,176.744938,15615.665390,6.138895,307.632511,366.680307,12.124105,55.952664,3.442915,0.000000
50%,7.027297,197.191839,20933.512750,7.143907,332.232177,423.455906,14.322019,66.542198,3.968177,0.000000
75%,8.052969,216.441070,27182.587067,8.109726,359.330555,482.373169,16.683049,77.291925,4.514175,1.000000
max,14.000000,317.338124,56488.672413,13.127000,481.030642,753.342620,27.006707,124.000000,6.494749,1.000000


In [32]:
# copy the data
data = data.copy()

# apply normalization techniques
for column in data.columns:
	data[column] = (data[column] - data[column].min()) / (data[column].max() - data[column].min())	


data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000,2011.000000
mean,0.497984,0.502267,0.384500,0.489383,0.580133,0.407644,0.490098,0.500973,0.499476,0.403282
std,0.114238,0.133835,0.153865,0.135038,0.117050,0.146292,0.134035,0.139289,0.154685,0.490678
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.425647,0.423434,0.272304,0.404565,0.507435,0.299173,0.400057,0.410452,0.395048,0.000000
50%,0.493723,0.507286,0.366982,0.490199,0.577314,0.402079,0.488659,0.502198,0.499168,0.000000
75%,0.568195,0.586226,0.478240,0.572493,0.654291,0.508867,0.583836,0.595331,0.607399,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[["ph", "Solids", "Chloramines", "Sulfate", "Conductivity", "Organic_carbon", "Trihalomethanes", "Turbidity"]], data["Potability"],test_size=0.2, random_state=42)

X_train.describe()

,ph,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000,1608.000000
mean,0.495831,0.386099,0.488709,0.579412,0.409220,0.489011,0.502027,0.498660
std,0.114489,0.152848,0.135388,0.117440,0.147064,0.131870,0.138823,0.154803
min,0.000000,0.015632,0.000000,0.000000,0.000000,0.087553,0.000000,0.000000
25%,0.421081,0.275069,0.401967,0.504780,0.299284,0.398866,0.411766,0.393748
50%,0.491522,0.369207,0.487935,0.575064,0.402521,0.487727,0.499472,0.499409
75%,0.567346,0.478600,0.571741,0.654112,0.513811,0.579762,0.594133,0.608488
max,0.952796,1.000000,0.992911,1.000000,1.000000,1.000000,1.000000,1.000000


In [34]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1608, 8)
(403, 8)
(1608,)
(403,)


In [42]:
inputs = keras.Input(shape=(8))
x = layers.Dense(16, activation="relu")(inputs)
x = layers.Dense(16, activation="relu")(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs = inputs, outputs = outputs)

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Nadam(learning_rate=0.1),
    metrics = ["accuracy"],
)

model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=2)
model.evaluate(X_test, y_test, batch_size=32, verbose=2) 

Epoch 1/20
51/51 - 1s - loss: 0.6803 - accuracy: 0.6014 - 753ms/epoch - 15ms/step
Epoch 2/20
51/51 - 0s - loss: 0.6757 - accuracy: 0.6026 - 72ms/epoch - 1ms/step
Epoch 3/20
51/51 - 0s - loss: 0.6745 - accuracy: 0.6026 - 71ms/epoch - 1ms/step
Epoch 4/20
51/51 - 0s - loss: 0.6746 - accuracy: 0.6026 - 83ms/epoch - 2ms/step
Epoch 5/20
51/51 - 0s - loss: 0.6750 - accuracy: 0.6026 - 71ms/epoch - 1ms/step
Epoch 6/20
51/51 - 0s - loss: 0.6753 - accuracy: 0.6026 - 75ms/epoch - 1ms/step
Epoch 7/20
51/51 - 0s - loss: 0.6749 - accuracy: 0.6026 - 75ms/epoch - 1ms/step
Epoch 8/20
51/51 - 0s - loss: 0.6737 - accuracy: 0.6026 - 70ms/epoch - 1ms/step
Epoch 9/20
51/51 - 0s - loss: 0.6766 - accuracy: 0.6026 - 77ms/epoch - 2ms/step
Epoch 10/20
51/51 - 0s - loss: 0.6752 - accuracy: 0.6026 - 90ms/epoch - 2ms/step
Epoch 11/20
51/51 - 0s - loss: 0.6761 - accuracy: 0.6026 - 82ms/epoch - 2ms/step
Epoch 12/20
51/51 - 0s - loss: 0.6759 - accuracy: 0.6026 - 83ms/epoch - 2ms/step
Epoch 13/20
51/51 - 0s - loss: 0.67

[0.6893266439437866, 0.5732010006904602]